# Actividad 5:
# Comparación Práctica de Métodos de Optimización en Regresión Lineal

---

> **Importante:** Este notebook debe ejecutarse en el orden sugerido para evitar errores. Algunas funciones dependen de bloques anteriores y podrían no estar disponibles si se omite su ejecución previa.

---

## Estructura:
1. Decisiones de diseño.
2. Funciones definidas.
3. Uso de las funciones y resultados.
4. Análisis de resultados y reflexiones.

---

## Decisiones de diseño

---

Consistió en implementar y comparar tres algoritmos de optimización para resolver un problema de regresión lineal simple utilizando datos sintéticos generados artificialmente.

Se realizaron los siguientes pasos:

- Generación de datos sintéticos:
Se simularon 100 observaciones con una relación lineal, lo que permite tener un escenario controlado y reproducible.

- Definición de la función de costo:
Se utilizó el error cuadrático medio (MSE) como métrica de evaluación del desempeño del modelo.

- Cálculo del gradiente:
Se derivó la función de costo respecto a los parámetros, obteniendo así las expresiones necesarias para los algoritmos de optimización.

- Aplicación de tres métodos de optimización:
    - **GD (Gradiente Descendente):** Usa todo el conjunto de datos por iteración.
    - **SGD (Estocástico):** Usa un solo punto aleatorio en cada paso.
    - **Adam:** Combina momentum y tasas de aprendizaje adaptativas.

- Visualización y comparación de resultados:
Se graficó la evolución del error (MSE) y la trayectoria de los parámetros para cada optimizador, además de mostrar los resultados en un DataFrame de pandas para comparar los tres métodos.

---

# Funciones definidas

---

Este proyecto se estructura en siete funciones principales, las cuales fueron divididas en tres bloques:

**Bloque 1: Generación de Datos y Funciones Base**

- **`generar_datos_sinteticos()`**  
  Genera un conjunto reproducible de datos sintéticos para regresión lineal, con ruido gaussiano añadido.

- **`error_cuadratico_medio()`**  
  Calcula el error cuadrático medio (MSE) entre las predicciones del modelo lineal y los valores reales.

- **`gradientes_mse()`**  
  Calcula los gradientes de la función de costo MSE con respecto a los parámetros pendiente y sesgo.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Generación de Datos Sintéticos
def generar_datos_sinteticos(n_muestras=100, semilla=42):
    """
    Genera datos sintéticos para un problema de regresión lineal simple con ruido gaussiano.

    Args:
        n_muestras (int): Número de muestras a generar.
        semilla (int): Semilla para reproducibilidad de los datos.

    Returns:
        x (np.ndarray): Variable independiente, valores entre 0 y 10.
        y (np.ndarray): Variable dependiente con ruido.
        w_real (float): Valor real de la pendiente del modelo.
        b_real (float): Valor real del sesgo del modelo.
    """
    np.random.seed(semilla)
    x = np.random.uniform(0, 10, n_muestras) # # Generación de datos aleatorios uniformemente distribuidos entre 0 y 10
    w_real, b_real = 2.5, 1.0 # w = peso/weight, b = sesgo/bias
    ruido = np.random.normal(0, 1, n_muestras)
    y = w_real * x + b_real + ruido
    return x, y, w_real, b_real


# Función de Costo y Gradientes
def funcion_costo(w, b, x, y):
    """
    Calcula el Error Cuadrático Medio (MSE) entre las predicciones y los valores reales.

    Args:
        w (float): Pendiente actual del modelo.
        b (float): Sesgo actual del modelo.
        x (np.ndarray): Variable independiente.
        y (np.ndarray): Variable dependiente real.

    Returns:
        float: Valor del MSE para los parámetros dados.
    """
    y_pred = w * x + b
    return np.mean((y - y_pred) ** 2) 

def gradientes_mse(w, b, x, y):
    """
    Calcula los gradientes del Error Cuadrático Medio respecto a la pendiente y el sesgo.

    Args:
        w (float): Pendiente actual del modelo.
        b (float): Sesgo actual del modelo.
        x (np.ndarray): Variable independiente.
        y (np.ndarray): Variable dependiente real.

    Returns:
        grad_w (float): Gradiente del MSE respecto a la pendiente (∂MSE/∂w).
        grad_b (float): Gradiente del MSE respecto al sesgo (∂MSE/∂b).
    """
    n = len(x)
    error = y - (w * x + b)
    grad_w = (-2 / n) * np.sum(x * error)
    grad_b = (-2 / n) * np.sum(error)
    return grad_w, grad_b

**Bloque 2: Métodos de Optimización**

- **`descenso_gradiente()`, `descenso_gradiente_estocastico()` y `optimizador_adam()`**  
  Implementan tres métodos de optimización para minimizar la función de costo: descenso de gradiente batch, descenso de gradiente estocástico y Adam, respectivamente.

> Cabe destacar que el optimizador **Adam fue implementado manualmente** (sin usar librerías como PyTorch) para mantener la coherencia estructural del código base y facilitar su integración con las demás funciones. Esta decisión se tomó especialmente porque Adam fue el último método en añadirse al proyecto debido a que era opcional, y rehacer el flujo completo con un framework externo podría haber requerido reescrituras innecesarias.

In [ ]:
def descenso_gradiente(x, y, tasa_aprendizaje=0.01, iteraciones=1000):
    """
    Implementa el algoritmo de Descenso de Gradiente (GD) para optimizar parámetros de regresión lineal.

    Args:
        x (np.ndarray): Variable independiente.
        y (np.ndarray): Variable dependiente real.
        tasa_aprendizaje (float): Tasa de aprendizaje (learning rate).
        iteraciones (int): Número de iteraciones para el algoritmo.

    Returns:
        w (float): Valor final de la pendiente (weight) del modelo.
        b (float): Valor final del sesgo (bias) del modelo.
        historial (dict): Diccionario que almacena el historial de los valores de w, b y el error (MSE)
                          en cada iteración. Contiene las claves 'w', 'b' y 'error'.
    """
    w, b = 0.0, 0.0
    historial = {'w': [], 'b': [], 'error': []}
    for _ in range(iteraciones):
        grad_w, grad_b = gradientes_mse(w, b, x, y)
        w -= tasa_aprendizaje * grad_w
        b -= tasa_aprendizaje * grad_b
        error = funcion_costo(w, b, x, y)
        historial['w'].append(w)
        historial['b'].append(b)
        historial['error'].append(error)
    return w, b, historial


def descenso_gradiente_estocastico(x, y, tasa_aprendizaje=0.01, iteraciones=1000):
    """
    Implementa el algoritmo de Descenso de Gradiente Estocástico (SGD) para optimizar parámetros.

    Args:
        x (np.ndarray): Variable independiente.
        y (np.ndarray): Variable dependiente real.
        tasa_aprendizaje (float): Tasa de aprendizaje.
        iteraciones (int): Número de iteraciones.

    Returns:
        w (float): Valor final de la pendiente (weight) del modelo.
        b (float): Valor final del sesgo (bias) del modelo.
        historial (dict): Diccionario con el historial de los valores de w, b y error (MSE)
                          por iteración. Contiene las claves 'w', 'b' y 'error'.
    """
    w, b = 0.0, 0.0
    historial = {'w': [], 'b': [], 'error': []}
    n = len(x)
    for _ in range(iteraciones):
        i = np.random.randint(0, n)
        xi, yi = x[i], y[i]
        pred = w * xi + b
        grad_w = -2 * xi * (yi - pred)
        grad_b = -2 * (yi - pred)
        w -= tasa_aprendizaje * grad_w
        b -= tasa_aprendizaje * grad_b
        error = funcion_costo(w, b, x, y)
        historial['w'].append(w)
        historial['b'].append(b)
        historial['error'].append(error)
    return w, b, historial


def optimizador_adam(x, y, tasa_aprendizaje=0.01, iteraciones=1000, beta1=0.9, beta2=0.999, epsilon=1e-8):
    """
    Implementa el optimizador Adam para la regresión lineal.

    Args:
        x (np.ndarray): Variable independiente.
        y (np.ndarray): Variable dependiente real.
        tasa_aprendizaje (float): Tasa de aprendizaje.
        iteraciones (int): Número de iteraciones.
        beta1 (float): Parámetro de decaimiento para el primer momento.
        beta2 (float): Parámetro de decaimiento para el segundo momento.
        epsilon (float): Pequeño valor para evitar división por cero.

    Returns:
        w (float): Valor final de la pendiente (weight) del modelo.
        b (float): Valor final del sesgo (bias) del modelo.
        historial (dict): Diccionario que contiene el historial de los valores de w, b y el error (MSE)
                          en cada iteración. Claves: 'w', 'b' y 'error'.
    """
    w, b = 0.0, 0.0
    m_w, v_w, m_b, v_b = 0.0, 0.0, 0.0, 0.0
    historial = {'w': [], 'b': [], 'error': []}
    
    for t in range(1, iteraciones + 1):
        grad_w, grad_b = gradientes_mse(w, b, x, y)
        
        # Actualizar momentos de primer orden (momentum)
        m_w = beta1 * m_w + (1 - beta1) * grad_w
        m_b = beta1 * m_b + (1 - beta1) * grad_b
        
        # Actualizar momentos de segundo orden (RMSprop)
        v_w = beta2 * v_w + (1 - beta2) * (grad_w ** 2)
        v_b = beta2 * v_b + (1 - beta2) * (grad_b ** 2)
        
        # Corrección de sesgo para los momentos
        m_w_corr = m_w / (1 - beta1 ** t)
        m_b_corr = m_b / (1 - beta1 ** t)
        v_w_corr = v_w / (1 - beta2 ** t)
        v_b_corr = v_b / (1 - beta2 ** t)
        
        # Actualizar parámetros
        w -= tasa_aprendizaje * m_w_corr / (np.sqrt(v_w_corr) + epsilon)
        b -= tasa_aprendizaje * m_b_corr / (np.sqrt(v_b_corr) + epsilon)
        
        error = funcion_costo(w, b, x, y)
        historial['w'].append(w)
        historial['b'].append(b)
        historial['error'].append(error)
        
    return w, b, historial


**Bloque 3: Visualización**

- **`visualizar_resultados()`**  
  Grafica la evolución del error y la trayectoria de los parámetros a lo largo de las iteraciones para cada método, facilitando la comparación de su comportamiento.

In [ ]:
def visualizar_resultados(hist_gd, hist_sgd, hist_adam):
    """
    Grafica la evolución del error y la trayectoria de los parámetros para los métodos de optimización.

    Args:
        hist_gd (dict): Historial con parámetros y errores para Descenso de Gradiente.
        hist_sgd (dict): Historial para Descenso de Gradiente Estocástico.
        hist_adam (dict): Historial para optimizador Adam.
    """
    plt.figure(figsize=(14, 5))

    # Evolución del error
    plt.subplot(1, 2, 1)
    plt.plot(hist_gd['error'], label='Gradiente')
    plt.plot(hist_sgd['error'], label='Estocástico')
    plt.plot(hist_adam['error'], label='Adam')
    plt.xlabel('Iteración')
    plt.ylabel('Error cuadrático medio')
    plt.title('Evolución del error')
    plt.legend()

    # Trayectoria de parámetros
    plt.subplot(1, 2, 2)
    plt.plot(hist_gd['w'], hist_gd['b'], label='Gradiente')
    plt.plot(hist_sgd['w'], hist_sgd['b'], label='Estocástico')
    plt.plot(hist_adam['w'], hist_adam['b'], label='Adam')
    plt.xlabel('Pendiente (w)')
    plt.ylabel('Sesgo (b)')
    plt.title('Trayectoria de parámetros')
    plt.legend()
    plt.tight_layout()
    plt.savefig('comparacion_metodos.png')
    plt.show()

# Uso de funciones

---

Se aplican las funciones definidas y se muestran los resultados obtenidos.

In [ ]:
x, y, w_real, b_real = generar_datos_sinteticos()
iteraciones = 1000
# Para efectos prácticos, pueden probarse tasas distintas para cada modelo, aunque pareciera que 0.01 es la mejor en general.
tasas = {
    'GD': 0.01,
    'SGD': 0.01,
    'Adam': 0.01
}

w_gd, b_gd, hist_gd = descenso_gradiente(x, y, tasas['GD'], iteraciones)
w_sgd, b_sgd, hist_sgd = descenso_gradiente_estocastico(x, y, tasas['SGD'], iteraciones)
w_adam, b_adam, hist_adam = optimizador_adam(x, y, tasas['Adam'], iteraciones)

# Crear DataFrame de resultados incluyendo valores reales
df_resultados = pd.DataFrame({
    'Optimizador': ['Gradiente', 'Estocástico', 'Adam', 'Valor Real'],
    'Tasa de Aprendizaje': [tasas['GD'], tasas['SGD'], tasas['Adam'], None],
    'Iteraciones': [iteraciones, iteraciones, iteraciones, None],
    'Pendiente (w)': [w_gd, w_sgd, w_adam, w_real],
    'Intersección (b)': [b_gd, b_sgd, b_adam, b_real],
    'Error MSE': [hist_gd['error'][-1], hist_sgd['error'][-1], hist_adam['error'][-1], None]
})

df_resultados = df_resultados.round(4)

# Mostrar resultados
print("\n=== Resultados Finales por Optimizador (y valores reales) ===\n")
print(df_resultados)

# Visualizar comparaciones
visualizar_resultados(hist_gd, hist_sgd, hist_adam)

# Análisis de Resultados y Reflexiones

---

## Resultados Finales

A continuación se muestran los resultados de los tres optimizadores aplicados a una regresión lineal con datos sintéticos, junto con los valores reales de la pendiente (`w`) e intersección (`b`):

| Optimizador | Tasa de Aprendizaje | Iteraciones | Pendiente (w) | Intersección (b) | Error MSE |
|-------------|---------------------|-------------|----------------|------------------|-----------|
| Gradiente   | 0.01                | 1000        | 2.45           | 1.21             | 0.81      |
| Estocástico | 0.01                | 1000        | 2.43           | 1.32             | 0.81      |
| Adam        | 0.01                | 1000        | 2.34           | 1.90             | 0.94      |
| **Valor Real** | -               | -           | **2.50**       | **1.00**         | -         |

---

## Evolución del Error (Gráfico Izquierdo)

- **Gradiente Descendente (GD)** y **Estocástico (SGD)** convergen rápidamente a un error cuadrático medio (MSE) bajo, estabilizándose antes de las 100 iteraciones.
- **Adam** muestra una convergencia más lenta. aunque su error sigue disminuyendo de forma gradual y no se estabiliza completamente hasta después de las ~300 iteraciones, tarda más en estabilizarse, alcanzando un MSE ligeramente superior a los otros métodos.

---

## Trayectoria de Parámetros (Gráfico Derecho)

- **GD** muestra una trayectoria estable y directa hacia los valores óptimos de los parámetros.
- **SGD** tiene una trayectoria mucho más ruidosa, con oscilaciones, lo cual es esperado por su naturaleza basada en ejemplos individuales.
- **Adam** presenta un avance más progresivo y estable, explorando el espacio de parámetros de forma controlada, aunque tarda más en llegar a la zona óptima.

---

## Reflexiones sobre Tasa de Aprendizaje y Métodos

- La **tasa de aprendizaje (α)** es un hiperparámetro crítico:
  - Muy alta → inestabilidad.
  - Muy baja → convergencia lenta o estancada.
- **GD** es ideal para conjuntos de datos pequeños y cuando se requiere precisión.
- **SGD** es eficiente para grandes volúmenes de datos, aunque ruidoso.
- **Adam** adapta la tasa de aprendizaje automáticamente, combinando lo mejor de GD y SGD, y suele ser robusto en tareas más complejas.

---

## Conclusión

Este ejercicio demuestra cómo un mismo problema de regresión puede dar resultados distintos dependiendo del optimizador y la tasa de aprendizaje usada. Ajustar correctamente estos parámetros es esencial para lograr eficiencia, estabilidad y precisión en el entrenamiento.

- Si bien con tasas de 0.01 los tres métodos mostraron buenos resultados, **cabe destacar que al utilizar tasas más pequeñas como 0.005 o 0.001, el optimizador Adam ve un deterioro significativo en su desempeño**, aumentando su MSE a más de 1 y hasta **80** respectivamente. Esto **evidencia la sensibilidad de cada optimizador a su tasa de aprendizaje**, y subraya la importancia crítica de ajustarla adecuadamente. Incluso modelos considerados robustos, como Adam, pueden fallar si los hiperparámetros no son elegidos con cuidado.

> En este caso, con una tasa de 0.01 y 1000 iteraciones, **todos los métodos alcanzaron un MSE menor a 1**, siendo **Adam el más lento en converger**, pero también el más suave en su evolución.